output folder name change to -> obj_fpfh_r0-5_random-drop or something.

In [1]:
import numpy as np
import os, sys, h5pyPCR
import open3d as o3d
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


INFO - 2022-11-04 07:58:40,044 - utils - NumExpr defaulting to 8 threads.


In [53]:
def drop_hole(pc, p):
    random_point = np.random.randint(0, pc.shape[0])
    index = np.linalg.norm(pc - pc[random_point].reshape(1,3), axis=1).argsort()
    return pc[index[int(pc.shape[0] * p):]]

In [54]:

# J: Aug #6
def multi_drop_hole(pc, n, p):
    """input:
        pc = point cloud
        n = number of holes
        p = percentage of points per hole that need to be taken out to make the hole
        
        Future work: Maybe I can add a random=true condition if true will randomly add 1-5 holes everytime it's called in data augmentation
    """
#     p = p/n # this makes holes with a relative percentage. 
    n_points = pc.shape[0]*p
    for i in range(n):
        random_point = np.random.randint(0, pc.shape[0])
        index = np.linalg.norm(pc - pc[random_point].reshape(1,3), axis=1).argsort()
        pc = pc[index[int(pc.shape[0] * p):]]
        p = n_points/pc.shape[0] # recalculate p to take out fixed number of points from each hole
    return pc

In [ ]:
#train
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

#test
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

In [ ]:
print(list(h5.keys()))
print(points.shape)

In [ ]:
def preprocess_point_cloud(pcd, voxel_size):
    # print(":: Downsample with a voxel size %.3f." % voxel_size)
    # pcd = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    # print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    # print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    pcd_fpfh = np.transpose(np.array(pcd_fpfh.data))
    return pcd_fpfh

In [ ]:
# print('points start', points.shape)

# local_features = np.empty([points.shape[0], points.shape[1], 33])
# for i in range(points.shape[0]):

#     points_temp = copy.deepcopy(points[i])

#     if np.random.random() > 0.5:
#         scale_ratio = np.random.uniform(low=0.03, high=1.6)
#         # scale_ratio = np.expand_dims(np.tile(scale_ratio, 3), 0)
#         points_temp *= scale_ratio

#     if np.random.random() > 0.5:
#         ## 3
#         r_drop = np.random.uniform(low=0.5, high=0.9)
#         size = int(points_temp.shape[0] * (1-r_drop))
#         random_indices = np.random.choice(points_temp.shape[0],
#                                         size=size,
#                                         replace=False)
#         points_temp = points_temp[random_indices, :]

#     else:
#         p_drop = np.random.uniform(low=0.25, high=0.75)
#         points_temp = drop_hole(pc=points_temp, p=p_drop)

#     if np.random.random() > 0.5:
#         ## Jitter
#         sigma = np.random.uniform(low=0.01, high=0.02)
#         N_, C_ = points_temp.shape
#         clip = 0.2
#         points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)

#     else:
#         p = np.random.uniform(low=0.01, high=0.1)
#         sigma = 0.1
#         N_, C_ = points_temp.shape

#         N__ = int(N_ * p)

#         # print('points_temp[:,0]', points_temp[:,0].shape)
#         x_min = np.min(points_temp[:,0])
#         x_max = np.max(points_temp[:,0])
#         y_min = np.min(points_temp[:,1])
#         y_max = np.max(points_temp[:,1])
#         z_min = np.min(points_temp[:,2])
#         z_max = np.max(points_temp[:,2])
#         noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
#         points_temp = np.vstack((points_temp, noise_points))


#     rand_indices = np.random.choice(points_temp.shape[0],
#                                     size=points[i].shape[0],
#                                     replace=True)
#     points_temp = points_temp[rand_indices, :]

#     # print('points   end', points_temp.shape)



#     pcd = o3d.geometry.PointCloud()
#     pcd.points = o3d.utility.Vector3dVector(points_temp)
#     pcd_fpfh = preprocess_point_cloud(pcd, 0.05)
#     local_features[i] = pcd_fpfh

In [ ]:
print('points start', points.shape)

local_features = np.empty([points.shape[0], points.shape[1], 33])
for i in range(points.shape[0]):

    points_temp =copy.deepcopy(points[i])

    
    # scale_ratio = 2
    # # scale_ratio = np.expand_dims(np.tile(scale_ratio, 3), 0)
    # points_temp *= scale_ratio


    # ## 3
    # r_drop = 0.9
    # size = int(points_temp.shape[0] * (1-r_drop))
    # random_indices = np.random.choice(points_temp.shape[0],
    #                                 size=size,
    #                                 replace=False)
    # points_temp = points_temp[random_indices, :]

    translate_ratio = np.array([0.2,0,0])
    points_temp += translate_ratio


    # p_drop = 0.7
    # points_temp = drop_hole(pc=points_temp, p=p_drop)


    # ## Jitter
    # sigma = 0.02
    # N_, C_ = points_temp.shape
    # clip = 0.2
    # points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)


    # p = 0.1
    # sigma = 0.1
    # N_, C_ = points_temp.shape

    # N__ = int(N_ * p)

    # # print('points_temp[:,0]', points_temp[:,0].shape)
    # x_min = np.min(points_temp[:,0])
    # x_max = np.max(points_temp[:,0])
    # y_min = np.min(points_temp[:,1])
    # y_max = np.max(points_temp[:,1])
    # z_min = np.min(points_temp[:,2])
    # z_max = np.max(points_temp[:,2])
    # noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    # points_temp = np.vstack((points_temp, noise_points))


    rand_indices = np.random.choice(points_temp.shape[0],
                                    size=points[i].shape[0],
                                    replace=True)
    points_temp = points_temp[rand_indices, :]

    # print('points   end', points_temp.shape)


    ## Calculate the local features FPFH
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_temp)
    pcd_fpfh = preprocess_point_cloud(pcd, 0.05)
    local_features[i] = pcd_fpfh

In [ ]:
print(local_features.shape)

In [ ]:
root = '/data/point_cloud/ScanObjectNN/robustness_created'
write_h5 = h5py.File(os.path.join(root, 'obj_fpfh_r0-5_aug', 'test_objectdataset_random_points_drop_0_9.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

In [ ]:
## load the saved file to check.

# root = '/data/point_cloud/ScanObjectNN/robustness_created'
# h5 = h5py.File(os.path.join(root, 'obj_fpfh_r0-5', 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)
# print(points.shape)

In [ ]:
## load the saved file to check.

# root = '/data/point_cloud/ScanObjectNN/robustness_created'
# h5 = h5py.File(os.path.join(root, 'obj_fpfh_r0-5', 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)
# print(points.shape)

In [ ]:
# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points[0])
# pcd_fpfh = preprocess_point_cloud(pcd, 0.05)
# local = np.transpose(np.array(pcd_fpfh.data))
# print('min', np.min(local))
# print('max', np.max(local))
# print('mean', np.mean(local))

Distance Local Features

In [ ]:
import os
import glob
in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed/*.h5') # list of all perturbed xyz H5 files

root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/'
for filepath in in_files:
    filename = os.path.basename(filepath)
    print(filename)

    h5 = h5py.File(os.path.join(root, 'xyz_perturbed', filename), 'r')
    points = np.array(h5['data']).astype(np.float32)
    labels = np.array(h5['label']).astype(int)
    h5.close()

    dist_features = np.empty([points.shape[0], points.shape[1], 33])
    for i in range(points.shape[0]):
        # print(i)
        points_temp = copy.deepcopy(points[i])
        
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(points_temp)

        pcd_tree = o3d.geometry.KDTreeFlann(pcd)
        for j in range(points_temp.shape[0]):
            [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[j], 34)

            idx = idx[1:]
            for l in range(len(idx)):
                dist_features[i,j,l] = np.sqrt(np.sum(np.square(points_temp[j] - points_temp[idx[l]])))
        
        dist_features[i,:,:] = (dist_features[i,:,:] - np.mean(dist_features[i,:,:]))/np.std(dist_features[i,:,:])
    

    # Write Test files
    write_h5 = h5py.File(os.path.join(root, 'dist_33', filename), 'w')
    write_h5.create_dataset("data", data = dist_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()

End of Distance features for perturbations

# ScanObject H5 to PCD

In [ ]:
# random scale
    # scale_ratio = np.array([0.3, 0.4, 0.5])
    # points_temp *= scale_ratio

    # scale
    # scale_ratio = 0.9
    # points_temp *= scale_ratio


    # p_drop = 0.25
    # points_temp = drop_hole(pc=points_temp, p=p_drop)

    ## -------------------
    # p = 0.5
    # sigma = 0.1
    # N_, C_ = points_temp.shape

    # N__ = int(N_ * p)

    # # print('points_temp[:,0]', points_temp[:,0].shape)
    # x_min = np.min(points_temp[:,0])
    # x_max = np.max(points_temp[:,0])
    # y_min = np.min(points_temp[:,1])
    # y_max = np.max(points_temp[:,1])
    # z_min = np.min(points_temp[:,2])
    # z_max = np.max(points_temp[:,2])
    # noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    # points_temp = np.vstack((points_temp, noise_points))
    ## -------------------

    # # Rotation
    # rotation_angle = 90
    # cosval = np.cos(rotation_angle)
    # sinval = np.sin(rotation_angle)
    # rotation_matrix = np.array([[cosval, 0, sinval],
    #                             [0, 1, 0],
    #                             [-sinval, 0, cosval]])
    # R = rotation_matrix.astype(np.float32)
    # points_temp = np.matmul(points_temp, R)

    # drop one patch/portion
    # p_drop = 0.75
    # points_temp = drop_hole(pc=points_temp, p=p_drop)
    ### ---------------------------------

    # sigma = 0.02
    # N_, C_ = points_temp.shape
    # clip = 0.2
    # points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)

    ### ---------------------------------
    ## 6 Multiple holes in the PC
    # points_temp = multi_drop_hole(pc=points_temp, n=3, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=4, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=5, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=6, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=7, p=0.1)

    ### ---------------------------------
    # ## --------------------------------------
    # # ## 3
    # size = int(points_temp.shape[0] * (1-.8))
    # random_indices = np.random.choice(points_temp.shape[0],
    #                                 size=size,
    #                                 replace=False)
    # points_temp = points_temp[random_indices, :]
    # # print('point_cloud after #3', point_cloud.shape)


        ### ---------------------------------
    # # # Jitter
    # sigma = 0.02
    # N_, C_ = points_temp.shape
    # clip = 0.2
    # points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)

    ## ------------------------------
    # add cluster of random points 

    # num_cluster = 15
    # N__ = 200 #add 200 points in each cluster

    # cloud_v1 = o3d.geometry.PointCloud()
    # cloud_v1.points = o3d.utility.Vector3dVector(points_temp)

    # pcd_tree = o3d.geometry.KDTreeFlann(cloud_v1)

    # rand_idx_cluster = np.random.choice(points_temp.shape[0],
    #                                     size=num_cluster,
    #                                     replace=False)

    # for i in range(len(rand_idx_cluster)):
    #     [k, idx_tree, _] = pcd_tree.search_knn_vector_3d(cloud_v1.points[rand_idx_cluster[i]], 20)
    #     points_small = points_temp[idx_tree]

    #     # print('points_temp[:,0]', points_temp[:,0].shape)
    #     x_min = np.min(points_small[:,0])
    #     x_max = np.max(points_small[:,0])
    #     y_min = np.min(points_small[:,1])
    #     y_max = np.max(points_small[:,1])
    #     z_min = np.min(points_small[:,2])
    #     z_max = np.max(points_small[:,2])            

    #     noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    #     points_temp = np.vstack((points_temp, noise_points))

    #     ### Do replace=False since number of points are more than original






In [3]:
# # train
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

#test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [4]:
# root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed/test_objectdataset_add_rand_points_group_15.h5'
# h5 = h5py.File(root, 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

In [ ]:
# drop one patch/portion
    # p_drop = 0.75
    # points_temp = drop_hole(pc=points_temp, p=p_drop)

    
    ## -------------------
    # # Add Random Points (Noise: inlier/outlier)
    # p = 0.1
    # sigma = 0.1
    # N_, C_ = points_temp.shape

    # N__ = int(N_ * p)

    # # print('points_temp[:,0]', points_temp[:,0].shape)
    # x_min = np.min(points_temp[:,0])
    # x_max = np.max(points_temp[:,0])
    # y_min = np.min(points_temp[:,1])
    # y_max = np.max(points_temp[:,1])
    # z_min = np.min(points_temp[:,2])
    # z_max = np.max(points_temp[:,2])
    # noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    # points_temp = np.vstack((points_temp, noise_points))



    ### ---------------------------------
    ## 6 Multiple holes in the PC
    # points_temp = multi_drop_hole(pc=points_temp, n=3, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=4, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=5, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=6, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=7, p=0.1)

    ## -------------------

    # size = int(points_temp.shape[0] * (1-.8))
    # random_indices = np.random.choice(points_temp.shape[0],
    #                                 size=size,
    #                                 replace=False)
    # points_temp = points_temp[random_indices, :]

    ## ------------------------------
    # # add cluster of random points 

    # num_cluster = 35
    # N__ = 200 #add 200 points in each cluster

    # cloud_v1 = o3d.geometry.PointCloud()
    # cloud_v1.points = o3d.utility.Vector3dVector(points_temp)

    # pcd_tree = o3d.geometry.KDTreeFlann(cloud_v1)

    # rand_idx_cluster = np.random.choice(points_temp.shape[0],
    #                                     size=num_cluster,
    #                                     replace=False)

    # for i in range(len(rand_idx_cluster)):
    #     [k, idx_tree, _] = pcd_tree.search_knn_vector_3d(cloud_v1.points[rand_idx_cluster[i]], 20)
    #     points_small = points_temp[idx_tree]

    #     # print('points_temp[:,0]', points_temp[:,0].shape)
    #     x_min = np.min(points_small[:,0])
    #     x_max = np.max(points_small[:,0])
    #     y_min = np.min(points_small[:,1])
    #     y_max = np.max(points_small[:,1])
    #     z_min = np.min(points_small[:,2])
    #     z_max = np.max(points_small[:,2])            

    #     cluster_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    #     points_temp = np.vstack((points_temp, cluster_points))

    #     ### Do replace=False since number of points are more than original


    ## HERE
    ### ---------------------------------

    

In [84]:
print('points start', points.shape)
# convert npy to pcd
out_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate_rotation_multi_patch_random_noise_4'

# local_features = np.empty([points.shape[0], points.shape[1], 33])
for i in range(points.shape[0]):

    # out_file_path = os.path.join(out_folder_path, str(i) + '_train' + ".pcd")
    out_file_path = os.path.join(out_folder_path, str(i) + '_test' + ".pcd")

    points_temp = copy.deepcopy(points[i])

    # # translate
    translate_ratio = 1.0
    points_temp += translate_ratio

    # # # # translate
    # translate_ratio = np.array([0,0,1.0])
    # points_temp += translate_ratio


    # # # Rotation
    rotation_angle = 90
    pcd_temp = o3d.geometry.PointCloud()
    pcd_temp.points = o3d.utility.Vector3dVector(points_temp)
    R = pcd_temp.get_rotation_matrix_from_xyz((0, rotation_angle, 0))
    R = R.astype(np.float32)
    points_temp = np.matmul(points_temp, R)

    ## 6 Multiple holes in the PC
    # points_temp = multi_drop_hole(pc=points_temp, n=3, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=4, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=5, p=0.1)
    # points_temp = multi_drop_hole(pc=points_temp, n=6, p=0.1)
    points_temp = multi_drop_hole(pc=points_temp, n=7, p=0.1)

    ## -------------------------

    # # # Add Random Points (Noise: inlier/outlier)
    p = 0.1
    sigma = 0.1
    N_, C_ = points_temp.shape

    N__ = int(N_ * p)

    # print('points_temp[:,0]', points_temp[:,0].shape)
    x_min = np.min(points_temp[:,0])
    x_max = np.max(points_temp[:,0])
    y_min = np.min(points_temp[:,1])
    y_max = np.max(points_temp[:,1])
    z_min = np.min(points_temp[:,2])
    z_max = np.max(points_temp[:,2])
    noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    points_temp = np.vstack((points_temp, noise_points))

    ## --------------------
    if points_temp.shape[0] > points[i].shape[0]:
        rand_indices = np.random.choice(points_temp.shape[0],
                                        size=points[i].shape[0],
                                        replace=False)
        points_temp = points_temp[rand_indices, :]
    elif points_temp.shape[0] < points[i].shape[0]:
        rand_indices = np.random.choice(points_temp.shape[0],
                                        size=points[i].shape[0],
                                        replace=True)
        points_temp = points_temp[rand_indices, :]
    else:
        points_temp = points_temp

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_temp)

    o3d.io.write_point_cloud(out_file_path, pcd)
    
print('Done')

points start (581, 2048, 3)
Done


In [ ]:
# # list all the point clouds in npy format
# # use list as input in next step
# import glob
# import os

# in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/original_pcd_files/*.pcd')

# with open("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_train_test_pcd_list.txt", "w") as output:
#     for row in in_files:
#         output.write(str(row) + '\n')

# New: XYZ perturbation to h5

In [88]:
path = '/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2'
folders = os.listdir(path)
print(folders)

['translate3_0', 'translate3_1', 'translate3_2', 'translate3_3', 'translate3_4', 'translate4_0', 'translate4_1', 'translate4_2', 'translate4_3', 'translate4_4', 'translate5_0', 'translate5_1', 'translate5_2', 'translate5_3', 'translate5_4', 'rotation2_0', 'rotation2_1', 'rotation2_2', 'rotation2_3', 'rotation2_4', 'rotation3_0', 'rotation3_1', 'rotation3_2', 'rotation3_3', 'rotation3_4', 'rotation4_0', 'rotation4_1', 'rotation4_2', 'rotation4_3', 'rotation4_4', 'rotation5_0', 'rotation5_1', 'rotation5_2', 'rotation5_3', 'rotation5_4', 'rotation6_0', 'rotation6_1', 'rotation6_2', 'rotation6_3', 'rotation6_4', 'translate_rotation_0', 'translate_rotation_1', 'translate_rotation_2', 'translate_rotation_3', 'translate_rotation_4', 'translate_multi_patch_0', 'translate_multi_patch_1', 'translate_multi_patch_2', 'translate_multi_patch_3', 'translate_multi_patch_4', 'translate_random_noise_0', 'translate_random_noise_1', 'translate_random_noise_2', 'translate_random_noise_3', 'translate_random

In [89]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d

for folder_name in folders:
    print(folder_name)
    in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/' + folder_name + '/*test.pcd')
    print(len(in_files))


    local_features = np.empty([len(in_files), 2048, 3], dtype='float32')
    in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/' + folder_name + '/'
    print('in_folder_path', in_folder_path)
    # in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/pcd_files/usc_pcd_files/'

    for i in range(len(in_files)):
        # print(i)
        file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

        pcd = o3d.io.read_point_cloud(file_path)
        cloud = np.array(pcd.points)

        # if i % 500 == 0:
        #     print(i)
        #     print('cloud.shape', cloud.shape)
        
        local_features[i,:,:] = cloud
        # print('cloud.shape', cloud.shape)
        # break
    print(local_features.shape)


    # #test
    root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed'
    h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
    # points = np.array(h5['data']).astype(np.float32)
    labels = np.array(h5['label']).astype(int)


    root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors'
    write_h5 = h5py.File(os.path.join(root, 'xyz_perturbed_ablation', 'test_objectdataset_' + folder_name + '.h5'), 'w')
    write_h5.create_dataset("data", data = local_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()


translate3_0
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate3_0/
(581, 2048, 3)
translate3_1
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate3_1/
(581, 2048, 3)
translate3_2
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate3_2/
(581, 2048, 3)
translate3_3
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate3_3/
(581, 2048, 3)
translate3_4
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate3_4/
(581, 2048, 3)
translate4_0
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate4_0/
(581, 2048, 3)
translate4_1
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/translate4_1/
(581, 2048, 3)
translate4_2
581
in_folder_path /data/point_clou

# local features to h5

In [2]:
path = '/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files'
folders = os.listdir(path)
print(folders)

['translate_rotation_multi_patch_random_noise_4', 'translate_rotation_multi_patch_random_noise_3', 'translate_rotation_multi_patch_random_noise_2', 'translate_rotation_multi_patch_random_noise_1', 'translate_rotation_multi_patch_random_noise_0', 'translate_rotation_multi_patch_4', 'translate_rotation_multi_patch_3', 'translate_rotation_multi_patch_2', 'translate_rotation_multi_patch_1', 'translate_rotation_multi_patch_0', 'translate_rotation_4', 'translate_rotation_3', 'translate_rotation_2', 'translate_rotation_1', 'translate_rotation_0', 'translate_random_noise_4', 'translate_random_noise_3', 'translate_random_noise_2', 'translate_random_noise_1', 'translate_random_noise_0', 'translate_multi_patch_4', 'translate_multi_patch_3', 'translate_multi_patch_2', 'translate_multi_patch_1', 'translate_multi_patch_0', 'translate5_4', 'translate5_3', 'translate5_2', 'translate5_1', 'translate5_0', 'translate4_4', 'translate4_3', 'translate4_2', 'translate4_1', 'translate4_0', 'translate3_4', 'tr

In [4]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d

for folder_name in folders:
    in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/' + folder_name + '/*test.pcd')
    print(len(in_files))


    local_features = np.empty([len(in_files), 2048, 33], dtype='float32')
    in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/' + folder_name + '/'
    print('in_folder_path', in_folder_path)
    # in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/pcd_files/usc_pcd_files/'

    for i in range(len(in_files)):
        # print(i)
        file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

        s = open(file_path).read()#.replace('-',' ')
        # print(s)
        # cloud = np.loadtxt(s, delimiter=' ')
        cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
        # cloud = cloud[:,:-9] # don't know what the last 9 values are. # for only 3dsc_1980_local and SHOT, USC

        # if i % 1000 == 0:
        #     print(i)
        #     print('cloud.shape', cloud.shape)
        
        local_features[i,:,:] = cloud
        # print('cloud.shape', cloud.shape)
        # break
    print(local_features.shape)


    # #test
    root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed'
    h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
    # points = np.array(h5['data']).astype(np.float32)
    labels = np.array(h5['label']).astype(int)


    root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors'
    write_h5 = h5py.File(os.path.join(root, 'fpfh_33_local_ablation', 'test_objectdataset_' + folder_name + '.h5'), 'w')
    write_h5.create_dataset("data", data = local_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()


581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/translate_rotation_multi_patch_random_noise_4/
(581, 2048, 33)
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/translate_rotation_multi_patch_random_noise_3/
(581, 2048, 33)
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/translate_rotation_multi_patch_random_noise_2/
(581, 2048, 33)
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/translate_rotation_multi_patch_random_noise_1/
(581, 2048, 33)
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/translate_rotation_multi_patch_random_noise_0/
(581, 2048, 33)
581
in_folder_path /data/point_cloud/ScanObjectNN/robustness

delete the following: xyz perturbation to h5 and local to h5

# XYZ perturbation to h5

In [4]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d

# absolute path to search all text files inside a specific folder
# path = 
in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/*test.pcd')
# in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/pcd_files/usc_pcd_files/*train.pcd')
print(len(in_files))
# print(in_files)

581


In [5]:
local_features = np.empty([len(in_files), 2048, 3], dtype='float32')
in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/'

for i in range(len(in_files)):
    # print(i)
    file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

    pcd = o3d.io.read_point_cloud(file_path)
    cloud = np.array(pcd.points)

    if i % 100 == 0:
        print(i)
        print('cloud.shape', cloud.shape)
    
    local_features[i,:,:] = cloud
    # print('cloud.shape', cloud.shape)
print(local_features.shape)

0
cloud.shape (2048, 3)
100
cloud.shape (2048, 3)
200
cloud.shape (2048, 3)
300
cloud.shape (2048, 3)
400
cloud.shape (2048, 3)
500
cloud.shape (2048, 3)
(581, 2048, 3)


In [6]:
# # train
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# # points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

# #test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [7]:
# ## Train
# root = '/data/point_cloud/ScanObjectNN/robustness_created'
# write_h5 = h5py.File(os.path.join(root, 'usc_1960_local', 'training_objectdataset_p1.h5'), 'w')
# write_h5.create_dataset("data", data = local_features)
# write_h5.create_dataset("label", data = labels[:1155])
# write_h5.close()

# Test
root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors'
write_h5 = h5py.File(os.path.join(root, 'xyz_perturbed', 'test_objectdataset_scale_0_9.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

local features to h5

In [8]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d

# absolute path to search all text files inside a specific folder
# path = 
in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/scale_0_9/*test.pcd')
# in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/pcd_files/usc_pcd_files/*train.pcd')
print(len(in_files))
# print(in_files)

581


In [9]:
local_features = np.empty([len(in_files), 2048, 33], dtype='float32')
# local_features = np.empty([1155, 2048, 33], dtype='float32')
in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/fpfh_perturbed_pcd_files/scale_0_9/'
# in_folder_path = '/data/point_cloud/ScanObjectNN/robustness_created/pcd_files/usc_pcd_files/'

for i in range(len(in_files)):
    # print(i)
    file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

    s = open(file_path).read()#.replace('-',' ')
    # print(s)
    # cloud = np.loadtxt(s, delimiter=' ')
    cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
    # cloud = cloud[:,:-9] # don't know what the last 9 values are. # for only 3dsc_1980_local and SHOT, USC

    if i % 100 == 0:
        print(i)
        print('cloud.shape', cloud.shape)
    
    local_features[i,:,:] = cloud
    # print('cloud.shape', cloud.shape)
    # break
print(local_features.shape)

0
cloud.shape (2048, 33)
100
cloud.shape (2048, 33)
200
cloud.shape (2048, 33)
300
cloud.shape (2048, 33)
400
cloud.shape (2048, 33)
500
cloud.shape (2048, 33)
(581, 2048, 33)


In [10]:
local = local_features[40]
print('min', np.min(local))
print('max', np.max(local))
print('mean', np.mean(local))

min 0.0
max 98.06431
mean 9.090909


In [11]:
# # train
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# # points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

# #test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [12]:
# ## Train
# root = '/data/point_cloud/ScanObjectNN/robustness_created'
# write_h5 = h5py.File(os.path.join(root, 'usc_1960_local', 'training_objectdataset_p1.h5'), 'w')
# write_h5.create_dataset("data", data = local_features)
# write_h5.create_dataset("label", data = labels[:1155])
# write_h5.close()

# Test
root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors'
write_h5 = h5py.File(os.path.join(root, 'fpfh_33_local', 'test_objectdataset_scale_0_9.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

delete later

In [ ]:
import glob
import os
import numpy as np
from io import StringIO

# absolute path to search all text files inside a specific folder
# path = 
in_files = glob.glob(r'/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/3dsc_pcd_files/*test.pcd')
print(len(in_files))
# print(in_files)


In [ ]:
local_features = np.empty([len(in_files), 2048, 1980], dtype='float32')
in_folder_path = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/3dsc_pcd_files/'
for i in range(len(in_files)):
    print(i)
    file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

    s = open(file_path).read()#.replace('-',' ')
    # print(s)
    cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
    cloud = cloud[:,:-9] # don't know what the last 9 values are

    # print(cloud.shape)
    
    local_features[i,:,:] = cloud
print(local_features.shape)

In [ ]:
#test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [ ]:

# Test
root = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created'
write_h5 = h5py.File(os.path.join(root, '3dsc_1980_local', 'test_objectdataset.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

In [ ]:
import glob
import os
import numpy as np
from io import StringIO

# absolute path to search all text files inside a specific folder
# path = 
in_files = glob.glob(r'/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/usc_pcd_files/*train.pcd')
print(len(in_files))
# print(in_files)

In [ ]:
local_features = np.empty([len(in_files), 2048, 1960], dtype='float16')
in_folder_path = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/usc_pcd_files/'
for i in range(len(in_files)):
    print(i)
    file_path = os.path.join(in_folder_path, str(i) + '_train' + ".pcd")

    s = open(file_path).read()#.replace('-',' ')
    # print(s)
    cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
    cloud = cloud[:,:-9] # don't know what the last 9 values are

    # print(cloud.shape)
    
    local_features[i,:,:] = cloud
print(local_features.shape)

In [ ]:
# train
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [ ]:
## Train
root = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created'
write_h5 = h5py.File(os.path.join(root, 'usc_1960_local', 'training_objectdataset.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

In [ ]:
import glob
import os
import numpy as np
from io import StringIO

# absolute path to search all text files inside a specific folder
# path = 
in_files = glob.glob(r'/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/usc_pcd_files/*test.pcd')
print(len(in_files))
# print(in_files)

In [ ]:
local_features = np.empty([len(in_files), 2048, 1960], dtype='float32')
in_folder_path = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created/usc_pcd_files/'
for i in range(len(in_files)):
    print(i)
    file_path = os.path.join(in_folder_path, str(i) + '_test' + ".pcd")

    s = open(file_path).read()#.replace('-',' ')
    # print(s)
    cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
    cloud = cloud[:,:-9] # don't know what the last 9 values are

    # print(cloud.shape)
    
    local_features[i,:,:] = cloud
print(local_features.shape)

In [ ]:

#test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [ ]:
# Test
root = '/home/jibanul/research/pc_robustness/data/ScanObjectNN/robustness_created'
write_h5 = h5py.File(os.path.join(root, 'usc_1960_local', 'test_objectdataset.h5'), 'w')
write_h5.create_dataset("data", data = local_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

Manual way of distance feature calculation for perturbation

In [ ]:
# # train
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
# h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
# labels = np.array(h5['label']).astype(int)

##test
# root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [ ]:
dist_features = np.empty([points.shape[0], points.shape[1], 33])
for i in range(points.shape[0]):
    print(i)
    points_temp = copy.deepcopy(points[i])

    # if np.random.random() > 0.5:
    scale_ratio = 0.9
    # scale_ratio = np.expand_dims(np.tile(scale_ratio, 3), 0)
    points_temp *= scale_ratio

    # if np.random.random() > 0.5:
    #     ## 3
    #     r_drop = np.random.uniform(low=0.5, high=0.9)
    #     size = int(points_temp.shape[0] * (1-r_drop))
    #     random_indices = np.random.choice(points_temp.shape[0],
    #                                     size=size,
    #                                     replace=False)
    #     points_temp = points_temp[random_indices, :]

    # else:
    #     p_drop = np.random.uniform(low=0.25, high=0.75)
    #     points_temp = drop_hole(pc=points_temp, p=p_drop)

    # if np.random.random() > 0.5:
    #     ## Jitter
    #     sigma = np.random.uniform(low=0.01, high=0.02)
    #     N_, C_ = points_temp.shape
    #     clip = 0.2
    #     points_temp += np.clip(sigma * np.random.randn(N_, C_), -1*clip, clip)

    # else:
    # p = 0.02
    # sigma = 0.1
    # N_, C_ = points_temp.shape

    # N__ = int(N_ * p)

    # # print('points_temp[:,0]', points_temp[:,0].shape)
    # x_min = np.min(points_temp[:,0])
    # x_max = np.max(points_temp[:,0])
    # y_min = np.min(points_temp[:,1])
    # y_max = np.max(points_temp[:,1])
    # z_min = np.min(points_temp[:,2])
    # z_max = np.max(points_temp[:,2])
    # noise_points = np.vstack((np.random.uniform(x_min, x_max, N__), np.random.uniform(y_min, y_max, N__), np.random.uniform(z_min, z_max, N__))).transpose().astype(np.float32)
    # points_temp = np.vstack((points_temp, noise_points))


    # rand_indices = np.random.choice(points_temp.shape[0],
    #                                 size=points[i].shape[0],
    #                                 replace=True)
    # points_temp = points_temp[rand_indices, :]

    
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points_temp)

    pcd_tree = o3d.geometry.KDTreeFlann(pcd)
    for j in range(points_temp.shape[0]):
        [k, idx, _] = pcd_tree.search_knn_vector_3d(pcd.points[j], 34)

        idx = idx[1:]
        for l in range(len(idx)):
            dist_features[i,j,l] = np.sqrt(np.sum(np.square(points_temp[j] - points_temp[idx[l]])))
    
    dist_features[i,:,:] = (dist_features[i,:,:] - np.mean(dist_features[i,:,:]))/np.std(dist_features[i,:,:])

In [ ]:
# ## Train
# root = '/data/point_cloud/ScanObjectNN/robustness_created'
# write_h5 = h5py.File(os.path.join(root, 'dist_33', 'training_objectdataset.h5'), 'w')
# write_h5.create_dataset("data", data = dist_features)
# write_h5.create_dataset("label", data = labels)
# write_h5.close()

# Test
root = '/data/point_cloud/ScanObjectNN/robustness_created'
write_h5 = h5py.File(os.path.join(root, 'dist_33', 'test_objectdataset_scale0_9.h5'), 'w')
write_h5.create_dataset("data", data = dist_features)
write_h5.create_dataset("label", data = labels)
write_h5.close()

Partial Views of XYZ and perturbations

In [13]:
import os
import glob
import open3d as o3d
import copy
import numpy as np

in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/*.pcd')

In [14]:
root = '/data/point_cloud/ScanObjectNN/robustness_created/partial_view_xyz_perturbed_pcd_files'

for file_path in in_files:
    print(file_path)
    pcd = o3d.io.read_point_cloud(file_path)

    # save partial view file
    out_file = file_path.split('.')[0].split('xyz_perturbed_pcd_files')[1]

    for i in range(120):
        pcd_r = copy.deepcopy(pcd)
        angle_x = np.random.uniform() * 2 * np.pi
        angle_y = np.random.uniform() * 2 * np.pi
        angle_z = np.random.uniform() * 2 * np.pi
        R = pcd_r.get_rotation_matrix_from_xyz((angle_x, angle_y, angle_z))
        pcd_r.rotate(R, center=(0, 0, 0))


        diameter = np.linalg.norm(
            np.asarray(pcd_r.get_max_bound()) - np.asarray(pcd_r.get_min_bound()))
            
        # print("Define parameters used for hidden_point_removal")
        camera = [0, 0, diameter]
        radius = diameter * 100

        # print("Get all points that are visible from given view point")
        _, pt_map = pcd_r.hidden_point_removal(camera, radius)

        # print("Visualize result")
        pcd_r = pcd_r.select_by_index(pt_map)

        out_file_path = root + out_file + '_partial_view_' + str(i) + '.pcd'
        o3d.io.write_point_cloud(out_file_path, pcd_r)


/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/0_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/1_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/2_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/3_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/4_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/5_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/6_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/7_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/8_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/9_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/

save original train and test files as well

In [15]:
import os
import glob
import open3d as o3d
import copy
import numpy as np

in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/*.pcd')

In [16]:
root = '/data/point_cloud/ScanObjectNN/robustness_created/partial_view_xyz_perturbed_pcd_files'

for file_path in in_files:
    print(file_path)
    pcd = o3d.io.read_point_cloud(file_path)

    # save partial view file
    out_file = file_path.split('xyz_perturbed_pcd_files')[1]

    out_file_path = root + out_file
    o3d.io.write_point_cloud(out_file_path, pcd)


/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/0_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/1_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/2_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/3_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/4_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/5_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/6_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/7_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/8_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/scale_0_9/9_test.pcd
/data/point_cloud/ScanObjectNN/robustness_created/

Create the txt file with file list for partial views

In [85]:
import os
import glob
import open3d as o3d
import copy
import numpy as np

in_files = glob.glob(r'/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files2/*/*.pcd')

In [87]:
with open("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_test_pcd_list_ablation.txt", "w") as output:
    for row in in_files:
        row = row.split('xyz_perturbed_pcd_files2')[1]
        output.write(str(row) + '\n')

Global Features

In [20]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d
train_test_list = np.loadtxt("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_train_test_pcd_list.txt", dtype=str)
test_list = np.loadtxt("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_test_pcd_list.txt", dtype=str)

In [21]:
pcd_root = '/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/esf_perturbed_pcd_files/'
h5_root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/esf_640_global'

In [22]:
import os
perturbation_list = []
for f in os.scandir(pcd_root):
    perturbation_list.append(f.name)

perturbation_list

['scale_0_9']

In [23]:
# #test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

for perturb_folder in perturbation_list:
    print(perturb_folder)

    in_folder_path = os.path.join(pcd_root, perturb_folder)

    global_features = np.empty([len(test_list), 121, 640], dtype='float32')
    i = 0
    for test_file in test_list:
        test_file = test_file.split('.')[0]
        temp_test_list = glob.glob(os.path.join(in_folder_path, test_file +'*')) # starts with original test cloud then the partial views are in order from 0-119

        cloud = []
        for temp_test_file in temp_test_list:
            s = open(temp_test_file).read()

            temp_cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
            cloud.append(temp_cloud)
        
        cloud = np.array(cloud)

        if i % 100 == 0:
            print(i)
            print('cloud.shape', cloud.shape)

        global_features[i,:,:] = cloud


        i += 1


    write_h5 = h5py.File(os.path.join(h5_root, 'test_objectdataset_' + perturb_folder + '.h5'), 'w')
    write_h5.create_dataset("data", data = global_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()

scale_0_9
0
cloud.shape (121, 640)
100
cloud.shape (121, 640)
200
cloud.shape (121, 640)
300
cloud.shape (121, 640)
400
cloud.shape (121, 640)
500
cloud.shape (121, 640)


train

In [ ]:
# #train
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

perturbation_list = ['original_pcd_files']

for perturb_folder in perturbation_list:
    print(perturb_folder)

    in_folder_path = os.path.join(pcd_root, perturb_folder)

    global_features = np.empty([2309, 121, 640], dtype='float32')
    for i in range(2309):
        # print(i)
        train_file = str(i) + '_train'
        temp_train_list = glob.glob(os.path.join(in_folder_path, train_file +'*')) # starts with original train cloud then the partial views are in order from 0-119

        cloud = []
        for temp_train_file in temp_train_list:
            s = open(temp_train_file).read()

            temp_cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
            cloud.append(temp_cloud)
        
        cloud = np.array(cloud)

        if i % 100 == 0:
            print(i)
            print('cloud.shape', cloud.shape)

        global_features[i,:,:] = cloud


    write_h5 = h5py.File(os.path.join(h5_root, 'training_objectdataset' + '.h5'), 'w')
    write_h5.create_dataset("data", data = global_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()

# Global features resample

In [24]:
import glob
import os
import numpy as np
import copy
import h5py

h5_root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/esf_640_global'
h5_root_out = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/esf_640_global_2048'

In [25]:
in_files = glob.glob(os.path.join(h5_root, '*scale_0_9.h5'))
# in_files = in_files[-3:]
in_files

['/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/esf_640_global/test_objectdataset_scale_0_9.h5']

In [26]:
npoints = 2048

# in_files = glob.glob(os.path.join(h5_root, '*.h5'))
for file_path in in_files:
    print(file_path)
    h5 = h5py.File(os.path.join(file_path), 'r')
    points = np.array(h5['data']).astype(np.float32)
    labels = np.array(h5['label']).astype(int)

    print('points', points.shape)
    points_new = np.empty([points.shape[0], npoints, points.shape[2]], dtype='float32') # saved train esf as float16

    for i in range(points.shape[0]):
        points_temp =copy.deepcopy(points[i])
        rand_indices = np.random.choice(points_temp.shape[0],
                                    size=npoints,
                                    replace=True)
        points_temp = points_temp[rand_indices, :]
    
        points_new[i,:,:] = copy.deepcopy(points_temp)

        if i % 100 == 0:
            print(i)

    print(points_new.shape)

    ## save file
    out_name = file_path.split('/')[-1]
    write_h5 = h5py.File(os.path.join(h5_root_out, out_name), 'w')
    write_h5.create_dataset("data", data = points_new)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()
    h5.close()
    

/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/esf_640_global/test_objectdataset_scale_0_9.h5
points (581, 121, 640)
0
100
200
300
400
500
(581, 2048, 640)


# Local Features create h5 files

In [ ]:
import glob
import os
import numpy as np
from io import StringIO
import sys, h5py
import open3d as o3d
train_test_list = np.loadtxt("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_train_test_pcd_list.txt", dtype=str)
test_list = np.loadtxt("/home/jibanul/research/pc_robustness/prep_data/scanobjectnn_test_pcd_list.txt", dtype=str)

In [ ]:
pcd_root = '/data/point_cloud/ScanObjectNN/robustness_created/descriptor_perturbed_pcd_files/shot_perturbed_pcd_files/'
h5_root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local'

In [ ]:
import os
perturbation_list = []
for f in os.scandir(pcd_root):
    perturbation_list.append(f.name)

In [ ]:
# #test
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'test_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

for perturb_folder in perturbation_list:
    print(perturb_folder)

    in_folder_path = os.path.join(pcd_root, perturb_folder)

    local_features = np.empty([len(test_list), 2048, 352], dtype='float32')
    i = 0
    for test_file in test_list:
        test_file_path = os.path.join(in_folder_path, test_file)

        s = open(test_file_path).read()

        cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
        cloud = cloud[:,:-9]
        
        cloud = np.array(cloud)

        if i % 100 == 0:
            print(i)
            print('cloud.shape', cloud.shape)

        local_features[i,:,:] = cloud


        i += 1
        
    write_h5 = h5py.File(os.path.join(h5_root, 'test_objectdataset_' + perturb_folder + '.h5'), 'w')
    write_h5.create_dataset("data", data = local_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()

train

In [ ]:
# #train
root = '/data/point_cloud/ScanObjectNN/main_split_nobg'
h5 = h5py.File(os.path.join(root, 'training_objectdataset.h5'), 'r')
# points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

perturbation_list = ['original_pcd_files']

for perturb_folder in perturbation_list:
    print(perturb_folder)

    in_folder_path = os.path.join(pcd_root, perturb_folder)

    local_features = np.empty([2309, 2048, 352], dtype='float32')
    for i in range(2309):
        # print(i)
        train_file = str(i) + '_train.pcd'

        train_file_path = os.path.join(in_folder_path, train_file)

        s = open(train_file_path).read()

        cloud = np.loadtxt(StringIO(s), delimiter=' ', skiprows=11)
        cloud = cloud[:,:-9]
        
        cloud = np.array(cloud)

        if i % 100 == 0:
            print(i)
            print('cloud.shape', cloud.shape)

        local_features[i,:,:] = cloud

    write_h5 = h5py.File(os.path.join(h5_root, 'training_objectdataset' + '.h5'), 'w')
    write_h5.create_dataset("data", data = local_features)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()

### 

### 

SHOT fix the nan issue
Note didn't work so changed code in data loader and added np.nan_to_num(points, posinf = 0.)

In [1]:
import glob
import os
import numpy as np
import copy
import h5py

h5_root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local'
h5_root_out = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local_new'

In [2]:
in_files = glob.glob(os.path.join(h5_root, '*.h5'))
in_files = in_files[-4:]
in_files

['/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local/training_objectdataset.h5',
 '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local/test_objectdataset_add_rand_points_group_10.h5',
 '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local/test_objectdataset_add_rand_points_group_5.h5',
 '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local/test_objectdataset.h5']

In [3]:
# in_files = glob.glob(os.path.join(h5_root, '*.h5'))
for file_path in in_files:
    print(file_path)
    h5 = h5py.File(os.path.join(file_path), 'r')
    points = np.array(h5['data']).astype(np.float32)
    labels = np.array(h5['label']).astype(int)

    points = np.nan_to_num(points, posinf = 0.)

    ## save file
    out_name = file_path.split('/')[-1]
    write_h5 = h5py.File(os.path.join(h5_root_out, out_name), 'w')
    write_h5.create_dataset("data", data = points)
    write_h5.create_dataset("label", data = labels)
    write_h5.close()
    h5.close()
    

/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/shot_352_local/training_objectdataset.h5


: 

: 

In [1]:
# Check add_rand_points_group_15

pcd_root = "/data/point_cloud/ScanObjectNN/robustness_created/xyz_perturbed_pcd_files/add_rand_points_group_15/0_test.pcd"
h5_root = '/data/point_cloud/ScanObjectNN/robustness_created/h5_descriptors/xyz_perturbed/test_objectdataset_add_rand_points_group_15.h5'

In [2]:
import h5py
import numpy as np
from io import StringIO

h5 = h5py.File(h5_root, 'r')
points = np.array(h5['data']).astype(np.float32)
labels = np.array(h5['label']).astype(int)

In [3]:
s = open(pcd_root).read()
cloud = np.loadtxt(StringIO(pcd_root), delimiter=' ', skiprows=11)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbe in position 173: invalid start byte